In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from datetime import datetime
import pandas as pd

In [2]:
browser = Browser('chrome')

In [3]:
# Pull in setlist data

url = 'https://phish.net/setlists/phish/'
browser.visit(url)
html = browser.html
phish_soup = soup(html, 'html.parser')

In [4]:
# Pull in concert attendance and box office data

url2 = 'https://en.wikipedia.org/wiki/Phish_concert_tours_and_festivals#Box_office_score_data'
browser.visit(url2)
html2 = browser.html
tour_soup = soup(html2, 'html.parser')
    

In [51]:
# pd.read_html for the win!

table_dfs = pd.read_html('https://en.wikipedia.org/wiki/Phish_concert_tours_and_festivals#Box_office_score_data')
money_dfs = []

for table_df in table_dfs:
    if table_df.columns[-1] == 'Gross':
        money_dfs.append(table_df)
money_dfs

[   Date (1993)                      City                 Venue  \
 0       May 29    Salinas, United States   Laguna Seca Raceway   
 1       May 30    Salinas, United States   Laguna Seca Raceway   
 2  December 31  Worcester, United States  Centrum in Worcester   
 3        TOTAL                     TOTAL                 TOTAL   
 
                Attendance     Gross  
 0         20,000 / 20,000  $504,082  
 1         20,000 / 20,000  $504,082  
 2         14,581 / 14,581  $320,220  
 3  34,581 / 34,581 (100%)  $824,302  ,
    Date (1994)                          City  \
 0  December 28   Philadelphia, United States   
 1  December 29     Providence, United States   
 2  December 30  New York City, United States   
 3  December 31         Boston, United States   
 4        TOTAL                         TOTAL   
 
                                            Venue              Attendance  \
 0  Philadelphia Convention Hall and Civic Center         10,325 / 10,325   
 1               

In [56]:
money_dfs[0]

,Date (1993),City,Venue,Attendance,Gross
0,May 29,"Salinas, United States",Laguna Seca Raceway,"20,000 / 20,000","$504,082"
1,May 30,"Salinas, United States",Laguna Seca Raceway,"20,000 / 20,000","$504,082"
2,December 31,"Worcester, United States",Centrum in Worcester,"14,581 / 14,581","$320,220"
3,TOTAL,TOTAL,TOTAL,"34,581 / 34,581 (100%)","$824,302"


Now we begin scraping the setlist data!

In [ ]:
dates = phish_soup.find_all('span', class_='setlist-date')
dates

date_strings = [date.text[-11:] for date in dates]

cleaned_date_strings = [date.strip() for date in date_strings]
cleaned_date_strings[0:5]

In [ ]:
dates

In [ ]:
venues = phish_soup.find_all('div', class_='setlist-venue')
venues

venue_strings = [venue.text.strip() for venue in venues]
venue_strings[0:5]

In [ ]:
locations = phish_soup.find_all('div', class_='setlist-location')
locations

locations = [location.text.strip() for location in locations]

cities = [location.split(',')[0] for location in locations]
states = [location[-2:] for location in locations]

print(cities[0:5])
print(states[0:5])
    

In [ ]:
set_list_notes = phish_soup.find_all('div', class_='setlist-notes')
set_list_notes

In [ ]:
phish_p1_df = pd.DataFrame({
    'Date': cleaned_date_strings,
    'Venue': venue_strings,
    'City': cities,
    'State': states
})
phish_p1_df[-5:]